In [1]:
import pandas as pd
import numpy as np
from scipy import io
import os, errno
import glob

In [2]:
# consolidate the true trials tDetect and exploration time across ALL the data.

#direc1 = r'E:\Downloads\ManducaMultiSenseData\Step3'
direc1 = r'./dataFolders/Output/Step3_v3/'
files = glob.glob(direc1 + 'L*.csv')
# files = [file for file in os.listdir(direc1) if file.endswith('.csv')]
# print(files)


# load all the files with IRdtect data 
#L0.1_c-3_m45, L0.1_c-3_m46, L50_c-3_m50 all have their IR data stitched together so either manually stitch back together with code at bottom of script or use stitched step 4 files on lab computre
direc2 = "G:/My Drive/Mahad/Light Level Experiments/Data Output from Analysis/Step 4/"
IR_data_files = glob.glob(direc2 + '*.npy')

In [3]:
new_path = r'./dataFolders/Output/Step5_v3'

try:
    os.makedirs(new_path)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [4]:
for fname in files:
    df = pd.read_csv(fname)
   
    a,b, c, _ = os.path.basename(fname).split("_")
    name = a + "_" + b + "_" + c
   
    # pull out all the trueTrials and reset the index to default to get true trial #
    #trueTrials = df[df.label == 1].loc[:, ['In_Frame', 'Out_Frame']].reset_index(drop = True)
    
    trueTrials = df.loc[:,'In_Frame':'Out_Frame']
    
    
    # load IR files and get indices of all the frames where IRDetect == 1
    strng = name + '.mat'
    IR_detect_file = [file for file in IR_data_files if strng in file]

#     print(IR_detect_file[0])
    loadedIRdetect = np.load(IR_detect_file[0])
    indx = np.where(loadedIRdetect)
    indx = indx[0].reshape(-1)

    # for each true trial, find the first frame where IRDetect == True
    proboscisDetect = []
    discoveryTime = []
    for framein, frameout in zip(trueTrials.In_Frame, trueTrials.Out_Frame):
        allframes = [frame for frame in indx if framein < frame < frameout]
        if allframes == []:
            detect = np.nan
        else:
            detect = min(allframes)
        Time  = detect - framein
        proboscisDetect.append(detect)
        discoveryTime.append(Time)

    df1 = pd.DataFrame({'ProboscisDetect': proboscisDetect})
    df2 = pd.DataFrame({'DiscoveryTime' : discoveryTime})

    new_data = pd.concat([trueTrials, df1, df2], axis = 1)
    new_data.to_csv(new_path + "\\" + name + '_RawDataForExplorationTime.csv')